#### LEVEL 2: Identify broad-scale, physiologically-relevant changes in burst features and timing between control and drugged samples
    (1) Burst-Aligned IFR Matrices and Averaging
        - What does activity look like centered on burst peaks?
        - Are units consistently recruited in a pattern across bursts?
        - Can we compute an average firing pattern across multiple bursts?
            - Extract IFR matrices centered on each burst
            - Sort units by peak firing time in each burst
            - Average across bursts for a clean picture

#### LEVEL 3: Single Burst Raster and Unit Order
    - Which units fired during a single burst?
    - When did they peak?
    - What is their temporal order?
    - Can I visualize a clear activation sequence?
        - IFR matrix for a burst and sort units by peak firing rate within that window.

#### LEVEL 4: Relative Timing and Firing Order
    Goals:
        Analyze when each unit fires relative to the population burst peak across bursts
        Add temporal precision to burst structure metrics for rank order correlations and functional role identification.
            - How early or late does a unit fire?
            - Do some units always fire early? Are they consistent?
            - Can we detect sequential recruitment or backbone structures?
        Needs:
            A list of time differences between:
                Each unit's peak activity
                Population peak in the same burst
        Tasks:
            - Build histograms of activation order
            - Identify early-firing units
            - Compare temporal dispersion of units across bursts
        Ideas for expansion:
            - Group units by quartiles of firing latency
            - Track variability in firing latency across bursts
            - Identify scaffold units by consistently early peaks
            - Identify functional units by consistent temporal correlations across bursts

#### LEVEL 5: Pairwise and Burst-to-Burst Relationships
    - How similar are units to each other during bursts?
    - Do certain pairs co-activate reliably?
    - Do bursts follow similar patterns over time?
    - Can I detect structure across the network beyond global averages?
        Tasks:
            - Compute pairwise correlation (Pearson), burst similarity (cosine), and possibly latency matrices.
            - Incorporate dimensional reduction (PCA/t-SNE) and cluster analysis with rich burst metrics acquired above.
### Navigation tips for S3 and Kubernetes

Quickly list (`ls`) or copy (`cp`) .acqm.zip files to cwd:
```
for UUID in \
  2025-06-09-e-MaxTwo_D57_KOLF2.2J_SmitsMidbrain_6OHDA_T2_72hr/derived/kilosort2 \
  2025-06-10-e-MaxTwo_D58_KOLF2.2J_SmitsMidbrain_6OHDA_T2_D4/derived/kilosort2 \
  2025-06-11-e-MaxTwo_D59_KOLF2.2J_SmitsMidbrain_6OHDA_T2_D5/derived/kilosort2 \
  2025-06-12-e-MaxTwo_D60_KOLF2.2J_6OHDA_T2_D6/derived/kilosort2
do
  echo "Scanning $UUID..."
  aws --endpoint-url https://s3.braingeneers.gi.ucsc.edu s3api list-objects-v2 \
    --bucket braingeneers \
    --prefix ephys/${UUID}/ \
    --query "Contents[?ends_with(Key, '_acqm.zip')].[Key]" \
    --output text |
  while read KEY; do
    if [ -n "$KEY" ] && [ "$KEY" != "None" ]; then
      FILE=$(basename "$KEY")
      echo "Downloading $FILE..."
      aws --endpoint-url https://s3.braingeneers.gi.ucsc.edu s3 ls \
        s3://braingeneers/"$KEY" "./$FILE"
    fi
  done
done
```
FILE_TYPE=".h5"

for UUID in \
  2025-06-09-e-MaxTwo_D57_KOLF2.2J_SmitsMidbrain_6OHDA_T2_72hr/derived/kilosort2 \
  2025-06-10-e-MaxTwo_D58_KOLF2.2J_SmitsMidbrain_6OHDA_T2_D4/derived/kilosort2 \
  2025-06-11-e-MaxTwo_D59_KOLF2.2J_SmitsMidbrain_6OHDA_T2_D5/derived/kilosort2 \
  2025-06-12-e-MaxTwo_D60_KOLF2.2J_6OHDA_T2_D6/derived/kilosort2
do
  echo "Scanning $UUID for '*$FILE_TYPE'..."

  aws --endpoint-url https://s3.braingeneers.gi.ucsc.edu s3api list-objects-v2 \
    --bucket braingeneers \
    --prefix ephys/${UUID}/ \
    --query "Contents[?ends_with(Key, \`${FILE_TYPE}\`)].Key" \
    --output text |
  while read -r KEY; do
    if [ -n "$KEY" ]; then
      echo "Found: $KEY"
    fi
  done
done

FILE_TYPE="_.acqm.zip"

for UUID in \
  2025-06-09-e-MaxTwo_D57_KOLF2.2J_SmitsMidbrain_6OHDA_T2_72hr/derived/kilosort2 \
  2025-06-10-e-MaxTwo_D58_KOLF2.2J_SmitsMidbrain_6OHDA_T2_D4/derived/kilosort2 \
  2025-06-11-e-MaxTwo_D59_KOLF2.2J_SmitsMidbrain_6OHDA_T2_D5/derived/kilosort2 \
  2025-06-12-e-MaxTwo_D60_KOLF2.2J_6OHDA_T2_D6/derived/kilosort2
do
  echo "Scanning $UUID for files containing '$FILE_TYPE'..."

  aws --endpoint-url https://s3.braingeneers.gi.ucsc.edu s3api list-objects-v2 \
    --bucket braingeneers \
    --prefix ephys/${UUID}/ \
    --query 'Contents[].Key' \
    --output text |
  grep "$FILE_TYPE" |
  while read -r KEY; do
    if [ -n "$KEY" ]; then
      echo "Found: $KEY"
    fi
  done
done



Search by keyword in UUID:
```
aws --endpoint-url https://s3.braingeneers.gi.ucsc.edu s3 ls s3://braingeneers/ephys/ --recursive | grep e-MaxTwo_D57
```
Search for one UUID:
```
aws --endpoint-url https://s3.braingeneers.gi.ucsc.edu s3 ls s3://braingeneers/ephys/2025-06-09-e-MaxTwo_D57_KOLF2.2J_SmitsMidbrain_6OHDA_T2_72hr/ --recursive
```
List all files in a bucket:
```
aws --endpoint-url https://s3.braingeneers.gi.ucsc.edu s3api list-objects-v2 \
  --bucket braingeneers \
  --prefix ephys/ \
  --delimiter / \
  --query "CommonPrefixes[].Prefix" \
  --output text
```
View credentials:
`
grep 'aws_access_key_id' ~/.aws/credentials
grep 'aws_secret_access_key' ~/.aws/credentials
`
In Kubernetes, create a secret in my cluster named s3 credentials:
`
kubectl create secret generic s3-credentials \
  --from-literal=AWS_ACCESS_KEY_ID="ID123" \
  --from-literal=AWS_SECRET_ACCESS_KEY="KEY123"
 `
Confirm secret was created: `kubectl get secrets`
View what's inside: `kubectl get secret s3-credentials -o yaml`
Decode and read later: `kubectl get secret s3-credentials -o jsonpath="{.data.AWS_ACCESS_KEY_ID}" | base64 --decode`

Help commands in s3:
    High-level commands (ls, cp, sync): `aws s3 ls help`
    Low-level api commands(`list-objects-v2`, `get-object`): `aws s3api list-objects-v2 help`

List all commands in Kubernetes: `kubectl`
Info about a specific commmand: `kubectl <command> --help`
List of command-line options for all commands: `kubectl options`



In [ ]:
# --- Set up imports and paths ---
import sys
from pathlib import Path

# Set the project root so Python can find analysis_libs and others
project_root = Path("~/bioinformatics").expanduser().resolve()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Define the data directory
data_path = project_root / "data/extracted/maxtwo_run2"

# Import burst analysis tools
from burst_analysis.loading import SpikeDataLoader
from burst_analysis.detection import BurstDetection
from burst_analysis.plotting import plot_overlay_raster_population


In [ ]:
# -----------------------------
# 1. Load one dataset
# -----------------------------
data_path = Path("~/bioinformatics/data/extracted/maxone_run1").expanduser()
loader = SpikeDataLoader(data_path)
datasets = loader.load()

# Pick one dataset
key = list(datasets.keys())[0]
spike_data = datasets[key]

# -----------------------------
# 2. Configure and run detector
# -----------------------------
config = {
    "bin_size_ms": 4,
    "square_win_ms": 5,
    "gauss_win_ms": 30,
    "threshold_rms": 2.0,
    "min_dist_ms": 200,
    "min_merge_separation_ms": 2000,
    "burst_edge_fraction": 0.25
}

bd = BurstDetection(spike_data.train, fs=spike_data.metadata.get("fs", 10000), config=config)

result = bd.compute_population_rate_and_bursts()
times, smoothed, peaks, peak_times, bursts, burst_windows = result

# -----------------------------
# 3. Plot raster + population
# -----------------------------
plot_overlay_raster_population(
    trains=spike_data.train,
    times=times,
    smoothed=smoothed,
    bursts=bursts,
    dataset_label=key,
    time_range=(0, 60)
)
